## Chapter 12 - Binomial Trees

Date: 19-Dec-2015

Learning source:
    Textbook: John C. Hull (2012), "Options, Futures, and Other Derivatives"


Model assumption:
 - Arbitrage opportunities do not exist. So that the portfolio we are constructing does not have risk --> it earns the risk-free rate.

In [3]:
import numpy as np
import pandas as pd

# Construct the Riskless Portfolio

## A Numerical Example

Consider a portfolio consisting of:
 - a long position in $\Delta$ shares of the stock
 - A short position in one option
 
Suppose that the stock prices either moves up from \$20 to \$22, or moves down to \$18, and the strike price of the option is \$21, and the option is an european call.

Procedure:

1. To calculate $\Delta$ that makes the portfolio riskless:

 At t=1, the value of two outcomes (if stock moves up or down) are equal
$$ 22\Delta + 1 = 18\Delta + 0 $$
It turns out that $\Delta=0.25$.

1. Find out the value of the portfolio as t=1:
 
 $$v_{t=1} = 22 \cdot 0.25 - 1 = 4.5$$
 Note that because the portfolio is riskless, the value of the porfolio is the same in the two outcomes.

1. Work out the present value of the portfolio at t=0 (suppose risk-free rate is 5%):
 $$ v_{t=0} = 4.5 \cdot e^{-0.05 \cdot 3/12} = 4.367 $$
 
1. Solve for the option price at t=0:
 $$ S_0 \cdot \Delta - f = v_0 $$
 And it shows that $f=0.633$.
 

## A Generalization

Consider a stock whose current price is $S_0$ and an option on the stock whose current price is $f$.
Suppose that the option lasts for time $T$ and that during the life of the option, the stock can either move up to a new level $S_{0}u$ or move down to a lower level $S_{0}d$. The payoff from the option in an up-movement is $f_u$, and the payoff in a down-movement is $f_d$.

Following the 4-steps outlined earlier, solve for $f$.

1. Find $\Delta$ to make the portfolio riskless:
 $$ v_u = S_{0}u \cdot {\Delta} -f_u $$
 $$ v_d = S_{0}d \cdot {\Delta} -f_d $$
 
 And
 
 $$ \Delta = \frac {f_u - f_d}  {S_0 (u - d)} $$
 
2. Find the value of the portfolio at t=1:
 $$ v_u = \frac{df_u - uf_d}{u - d} $$
 
3. Find the present value of the portfolio
 $$ v_0 = e^{-rT} \cdot \frac{df_u - uf_d}{u - d} $$
 
4. Find the value of the option:
 $$ S_0 \Delta - f = v_0 = e^{-rT} \cdot (S_0 u \Delta - f_u) $$
 Substutite in $\Delta$:
 $$ f = e^{-rT} \cdot (p f_u + (1-p) f_d) $$
 where 
 $$ p = \frac {e^{rT} - d} {u - d}  or  \frac {S_0 e^{rT} - S_d} {S_u - S_d} $$ 
 And it measures the probability of an up-movement.

Final note: It is a bit surprising that the pricing formula does not involve the probabilities of the stock price moving up or down. The reason is that we are aclculating the option value in terms of the price of the underlying stock. The probabilities of future up or down movements are already incorporated into the stock price.

In [14]:
def binomial_tree_1_branch(s=10, s_u=11, s_d=9, x=10.5, t=0.25, rf=0.12):
    """
    Module: Calculate the option price for 1-step binomial tree
    Parameters are s_u, s_d, X, T, rf and s.
    s: Asset price at T=0
    s_u: Asset price at T=1 if price increases
    s_d: Asset price at T=1 if price decreases
    x: Stike price of the option
    t: Maturity of the option (in years)
    rf: Risk free rate (p.a. e.g. 0.12)
    Module returns f: the option price at T=0.
    Portfolio: Long Delta assets while short an option
    """

    # Find P
    p = ( s*np.exp(rf*t) - s_d ) / (s_u - s_d)

    # Find f
    f_u = np.max((0, s_u - x))
    f_d = np.max((0, s_d - x))
    f = ( p*f_u + (1 - p)*f_d ) * np.exp( - rf*t )

    print("Parameters are: S={s}, S_u={s_u}, S_d={s_d}, X={x}, T={t}, Rf={rf}.".format(s=s, s_u=s_u, s_d=s_d, x=x, t=t, rf=rf))
    print("Option price f = {f:.4f}, with probability of up-move p = {p:.4f}, f_u={f_u}, f_d={f_d}".format(f=f, p=p, f_u=f_u, f_d=f_d))

binomial_tree_1_branch()

Parameters are: S=10, S_u=11, S_d=9, X=10.5, T=0.25, Rf=0.12.
Option price f = 0.3165, with probability of up-move p = 0.6523, f_u=0.5, f_d=0.0


# Matching Volatility with u and d

In practice, the parameters $u$ and $d$ are chosen to match the volatility of the stock.

To begin with, just to borrow a formula from the Black-Scholes: The volatility of a stock price, $\sigma$, is defined so that $\sigma \sqrt{\Delta t}$ is the standard deviation of the return on the stock price in a short period of time of length $\Delta t$.

Example: 1-Branch Binomial Tree

Variance of the stock return $u-1$ and $d-1$ is the same as the $u$ and $d$.

$$ Var = E[ret^2] - [E(ret)]^2 
 = p^2 u^2 + (1-p)^2 d^2 - [p u + (1-p)d]^2
$$

To match the volatility, we must have:
$$ Var = \sigma^2 \Delta t $$

When terms in second and higher powers of $\Delta t$ are ignored, a solution proposed by Cox, Ross and Rubinstein (1979) is that:
$$ u = e^{\sigma \sqrt{\Delta t}} $$
$$ d = e^{- \sigma \sqrt{\Delta t}} $$


## Options on Stocks Paying a Continuous Dividend Yield

Consider a stock paying a known dividend yield at rate $q$. As a result of that, the capital gain of the stock investment is $r-q$, assuming full dividend reinvestment.

The new formula is that:
$$ p = \frac {e^{(r-q)T} - d} {u - d}   or  \frac {S_0 e^{(r-q)T} - S_d} {S_u - S_d} $$ 

# A Binomial-Tree in Python

In [38]:
# To calculate n-tree european/american option price
# By matching volatility of the stock

def node_price(price, n_up, n_down, u, d):
    """
    Find the price of n_up up-moves; and m_down down-moves;
    price: beginning stock price
    n_up: number of up-moves
    n_down: number of down_moves
    u: the up factor
    d: the down factor
    """
    node_price = price * u**n_up * d**n_down
    return node_price


def binomial_tree(k=0,
                  n_up=0,
                  price=20, 
                  volatility=0.3,
                  rf=0.05,
                  t=1,
                  x=21,
                  q=0,
                  steps=2,
                  call_or_put='call',
                  american=False,
                  options_on_futures=False):
    """
    Return the option price using a n-step binomial tree at k-th level of decision node with n-up up-movements.

    Parameters:
        k: the level of decision nodes (number of paths since t=0)
        n_up: among the k paths, the number of up-movements.
        price: price of the stock at t=0;
        volatility: i.e. 30% is expressed as 0.3; the volatility of the stock return series, or the standard deviation of return times the square root of delta t.
        rf: the risk free rate p.a.
        t: the time expiry of the option
        x: the strike price
        q: the dividend yield or (r_foreign: interest rate on foreign currency if option on currencies)
        steps: number of steps in the binomial tree
        call_or_put: Takes the value of 'call' or 'put'; defaulted to 'call'.
        american: True if American option, False if European option.
        options_on_futures: True if options on futures; in a risk-neutral world a futures price should have an expected growth rate of zero.


    Formulas:
        Assumed generating the binomial tree by matching the volatility of the stock prices:
        The up move factor:
            u = exp( sigma*sqrt(delta t) )
        The down move factor:
            d = exp( - sigma*sqrt(delta t) )

        The probability of an up-move:
            p = ( exp( (rf - q)*delta t ) - d ) / ( u - d )
    """
    
    delta_t = t/steps
    u = np.exp( volatility*np.sqrt(delta_t) )
    d = np.exp( - volatility*np.sqrt(delta_t) )
    
    if options_on_futures:
        p = (1- d) / (u - d)
    else:
        p = ( np.exp( (rf - q)*delta_t) - d ) / (u - d)
    
    n_down = k - n_up
    node_p = node_price(price, n_up, n_down, u, d)

    if call_or_put=='call':
        node_f = np.max((0, node_p - x))
    else:
        node_f = np.max((0, x - node_p))
        
    global disp_parameters
    if disp_parameters:
        print("""Parameters are:
        k [evaluation: the level of decision node you like to price the option at] = {k}
        n_up [evaluation: the number of up-movements out of {k} paths (to determin the particular node)] = {n_up}
        price [initial price of the stock at t=0] = {price}
        volatility [the volatility of the stock return] = {volatility}
        rf [risk free rate] = {rf}
        T [time expiry of the option] = {t}
        X [the strike price] = {x}
        q [the dividend yield] = {q}
        steps [number of steps in the binomial tree] = {steps}
        call_or_put = {call_or_put}
        american = {american}
        options_on_futures = {options_on_futures}
        delta_t = {delta_t}
        u = {u}
        d = {d}
        p = {p}
        S_t = {node_p}
        """.format(k=k, n_up=n_up, price=price, volatility=volatility, rf=rf,
                   t=t, x=x, q=q, steps=steps, call_or_put=call_or_put, american=american, 
                   options_on_futures=options_on_futures,
                   delta_t=delta_t, u=u, d=d, p=p, node_p=node_p))
        disp_parameters = False


    if k==steps: 
        # end nodes
        return node_f

    # Using forward induction to find intermediate nodes
    expected_node_f_fv = p * binomial_tree(k + 1, n_up + 1, price, volatility, rf, t, x, q, steps, call_or_put, american, options_on_futures) +\
                      (1 - p) * binomial_tree(k + 1, n_up, price, volatility, rf, t, x, q, steps, call_or_put, american, options_on_futures)
    expected_node_f = expected_node_f_fv*np.exp( -(rf - q)*delta_t )

    if american==True:
        return np.max((expected_node_f, node_f))
    else:
        return expected_node_f

disp_parameters = True
f = binomial_tree(k=0, 
                  n_up=0, 
                  price=810, 
                  volatility=0.2, 
                  q=0.02, 
                  rf=0.05, 
                  t=0.5, 
                  x=800, 
                  steps=2, 
                  call_or_put='call', 
                  american=False,
                  options_on_futures=False)
print("Option price: ${f:.3f}".format(f=f))


disp_parameters = True
f = binomial_tree(k=0, 
                  n_up=0, 
                  price=31, 
                  volatility=0.3,
                  q=0.00, 
                  rf=0.05, 
                  t=0.75, 
                  x=30, 
                  steps=3, 
                  call_or_put='put', 
                  american=True,
                  options_on_futures=True)
print("Option price: ${f:.3f}".format(f=f))

Parameters are:
        k [evaluation: the level of decision node you like to price the option at] = 0
        n_up [evaluation: the number of up-movements out of 0 paths (to determin the particular node)] = 0
        price [initial price of the stock at t=0] = 810
        volatility [the volatility of the stock return] = 0.2
        rf [risk free rate] = 0.05
        T [time expiry of the option] = 0.5
        X [the strike price] = 800
        q [the dividend yield] = 0.02
        steps [number of steps in the binomial tree] = 2
        call_or_put = call
        american = False
        options_on_futures = False
        delta_t = 0.25
        u = 1.1051709180756477
        d = 0.9048374180359595
        p = 0.5125991278953855
        S_t = 810.0
        
Option price: $53.931
Parameters are:
        k [evaluation: the level of decision node you like to price the option at] = 0
        n_up [evaluation: the number of up-movements out of 0 paths (to determin the particular node)] = 0

In [40]:
np.exp(0.08/12) - (38/40)
print(3*0.566*np.exp(-0.08/12))

1.68671764962
